In [3]:
from rollout import play_game, Network

In [4]:
network = Network('.')

In [7]:
%timeit game = play_game(network)

1.65 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
data = game.get_data()

Elementary operations to save / load the game dictionary

In [86]:
import numpy as np
import io
from tqdm import tqdm

# Save the data dict to a binary sequence
with io.BytesIO() as f:
    np.savez_compressed(f, **data)
    binary_data = f.getvalue()
    
del data

# Lead the data dict from a binary sequence
with io.BytesIO(binary_data) as f:
    data = dict(np.load(f, allow_pickle=True).items())

data['mol_smiles']

array(['C', 'C=O', 'O=C=O'], dtype='<U5')

In [66]:
import psycopg2

dbparams = {
    'dbname': 'bde',
    'port': 5432,
    'host': 'yuma.hpc.nrel.gov',
    'user': 'rlops',
    'password': '********',
    'options': f'-c search_path=rl',
}

    
with psycopg2.connect(**dbparams) as conn:
    with conn.cursor() as cur:
        cur.execute("""
        DROP TABLE IF EXISTS TestReplay;
        
        CREATE TABLE TestReplay (
            gameid serial PRIMARY KEY,
            time timestamp DEFAULT CURRENT_TIMESTAMP,
            data BYTEA);
            
        INSERT INTO TestReplay (data) VALUES (%s)
        """, (binary_data,))

We can use pandas to read out SQL queries as well

In [84]:
import pandas as pd

with psycopg2.connect(**dbparams) as conn:
    df = pd.read_sql_query("""
    SELECT * from TestReplay;
    """, conn)
    
df

,gameid,time,data
0,1,2020-08-31 05:41:17.018763,"[b'P', b'K', b'\x03', b'\x04', b'\x14', b'\x00..."


In [83]:
def read_game_data(binary_data):
    """Parse our binary game format. We can use pandas.apply to map 
    this over an entire column if we want """
    with io.BytesIO(df.iloc[0].data) as f:
        data = dict(np.load(f, allow_pickle=True).items())
        
    return data

df.data.apply(read_game_data)

0    {'network_inputs': {'mol': [array([0., 0., 0.,...
Name: data, dtype: object

In [89]:
def save_game_postgresql(game, conn):
    """ Just combines a bunch of the last few functions together """
    
    data = game.get_data()
    
    with io.BytesIO() as f:
        np.savez_compressed(f, **data)
        binary_data = f.getvalue()
        
    with conn:
        with conn.cursor() as cur:
            cur.execute("INSERT INTO TestReplay (data) VALUES (%s)",
                        (binary_data,))        

In [90]:
# I think it makes sense to keep this connection alive, 
# as long as the games are somewhat fast.. But we could certainly
# close it and re-open even 

with psycopg2.connect(**dbparams) as conn:
    for _ in tqdm(range(10)):
        game = play_game(network)
        save_game_postgresql(game, conn)

100%|██████████| 10/10 [00:22<00:00,  2.27s/it]


In [103]:
# Here's how we'd grab the most recent batch of games from the buffer

with psycopg2.connect(**dbparams) as conn:
    df = pd.read_sql_query("""
    SELECT * from TestReplay order by time desc limit 4;
    """, conn)

df

,gameid,time,data
0,11,2020-08-31 05:56:14.722273,"[b'P', b'K', b'\x03', b'\x04', b'\x14', b'\x00..."
1,10,2020-08-31 05:56:13.052142,"[b'P', b'K', b'\x03', b'\x04', b'\x14', b'\x00..."
2,9,2020-08-31 05:56:11.396527,"[b'P', b'K', b'\x03', b'\x04', b'\x14', b'\x00..."
3,8,2020-08-31 05:56:09.286064,"[b'P', b'K', b'\x03', b'\x04', b'\x14', b'\x00..."


This is a super basic version of how this might work, but we'd likely want to split out things like reward, final SMILES, etc as seperate columns perhaps. In case we wanted to use those later as selection criteria